# Preprocess the Employee Dataset

## This notebook is for Data Cleaning and Feature Engineering

**==============================================================================================================**

## Import Libraries

In [1]:
import numpy as np
#from numpy import count_nonzero, median, mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import squarify

import datetime
from datetime import datetime, timedelta, date, time

#import os
#import zipfile
import scipy
from scipy import stats
#from scipy.stats.mstats import normaltest # D'Agostino K^2 Test
#from scipy.stats import boxcox
from collections import Counter

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, Binarizer
from sklearn.impute import SimpleImputer, MissingIndicator, KNNImputer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import set_config

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

# Use Feature-Engine library
import feature_engine

from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, DropMissingData, MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer, RandomSampleImputer

from feature_engine.outliers import Winsorizer, ArbitraryOutlierCapper, OutlierTrimmer

from feature_engine.encoding import CountFrequencyEncoder, DecisionTreeEncoder, MeanEncoder, OneHotEncoder
from feature_engine.encoding import OrdinalEncoder, WoEEncoder, RareLabelEncoder, StringSimilarityEncoder

from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser, ArbitraryDiscretiser
from feature_engine.discretisation import DecisionTreeDiscretiser, EqualWidthDiscretiser

from feature_engine.datetime import DatetimeFeatures

from feature_engine.creation import CyclicalFeatures, MathFeatures, RelativeFeatures


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


**==============================================================================================================**

Join employee_hire_dates with employee_info using a full outer join and selecting the join clause based on the common fields from the two tables

In [2]:
empinfo = pd.read_csv("employee_info.csv")
empinfo.head()

,employee_id,cell_phone,city,state,num
0,6755.00,1-840-613-6475,Chicago,Illinois,60653.00
1,6885.00,1-912-845-2573,San Diego,California,92105.00
2,8471.00,1-144-282-6032,San Francisco,California,94122.00
3,7209.00,1-357-545-1640,Lafayette,Louisiana,70506.00
4,5644.00,1-286-161-5912,Peoria,Illinois,61604.00


In [3]:
empinfo = empinfo.rename(columns =  {'num': 'zipcode'})

In [4]:
empinfo.head()

,employee_id,cell_phone,city,state,zipcode
0,6755.00,1-840-613-6475,Chicago,Illinois,60653.00
1,6885.00,1-912-845-2573,San Diego,California,92105.00
2,8471.00,1-144-282-6032,San Francisco,California,94122.00
3,7209.00,1-357-545-1640,Lafayette,Louisiana,70506.00
4,5644.00,1-286-161-5912,Peoria,Illinois,61604.00


In [5]:
empinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   employee_id  73 non-null     float64
 1   cell_phone   73 non-null     object 
 2   city         73 non-null     object 
 3   state        73 non-null     object 
 4   zipcode      73 non-null     float64
dtypes: float64(2), object(3)
memory usage: 5.5+ KB


In [6]:
emphire = pd.read_csv("employee_hire_dates.csv")
emphire.head()

,id,hire date
0,6755.00,4/9/2017
1,6885.00,5/9/2015
2,8471.00,28/11/2014
3,7209.00,30/5/2017
4,5644.00,24/3/2015


In [7]:
emphire.shape

(131, 2)

In [8]:
df = pd.merge(left=empinfo, right=emphire, left_on="employee_id", right_on="id", how="outer")

In [9]:
df

,employee_id,cell_phone,city,state,zipcode,id,hire date
0,6755.00,1-840-613-6475,Chicago,Illinois,60653.00,6755.00,4/9/2017
1,6885.00,1-912-845-2573,San Diego,California,92105.00,6885.00,5/9/2015
2,8471.00,1-144-282-6032,San Francisco,California,94122.00,8471.00,28/11/2014
3,7209.00,1-357-545-1640,Lafayette,Louisiana,70506.00,7209.00,30/5/2017
4,5644.00,1-286-161-5912,Peoria,Illinois,61604.00,5644.00,24/3/2015
...,...,...,...,...,...,...,...
3974,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3975,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3977,NaN,NaN,NaN,NaN,NaN,7015.00,17/10/2016


In [10]:
empt1 = pd.read_csv("employee_travel_1.csv")
empt1.head()

,employee_id,date_entry,first_country,2nd_country,thrd_count
0,6755,2020-04-07 00:58:52,"Saint Helena, Ascension and Tristan da Cunha",Palau,Gibraltar
1,6885,2019-06-02 17:56:06,Georgia,Mongolia,Croatia
2,8471,2020-05-20 03:33:18,San Marino,Georgia,South Africa
3,7209,2019-07-31 01:42:43,Denmark,Iceland,Tonga
4,5644,2018-11-09 13:59:35,Bolivia,"Virgin Islands, United States",Greece


In [11]:
empt1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   employee_id    24 non-null     int64 
 1   date_entry     24 non-null     object
 2   first_country  24 non-null     object
 3   2nd_country    24 non-null     object
 4   thrd_count     24 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.1+ KB


In [12]:
empt2 = pd.read_csv("employee_travel_2.csv", encoding='latin-1')
empt2.head()

,employee_id,name,date_entry,first_country,2nd_country,thrd_count,favorite_word,cell_phone
0,8517,Guinevere Tate,2019-07-23 11:43:24,Ireland,Cayman Islands,Antarctica,"orci,",1-860-576-7072
1,8848,Damon Mcknight,2019-06-28 15:26:49,Norfolk Island,Fiji,Venezuela,ridiculus,1-119-624-7552
2,7927,Sopoline Hickman,2019-02-19 11:37:33,Switzerland,Brunei,Maldives,Aliquam,1-310-585-7169
3,7377,Carissa Glenn,2019-04-18 00:59:10,Finland,Reunion,Denmark,neque.,1-178-367-5761
4,8885,Micah Mckinney,2019-04-06 04:20:35,Turks and Caicos Islands,Ireland,Singapore,fringilla,1-250-235-7397


In [13]:
empt3 = pd.read_csv("employee_travel_3.csv", encoding='latin-1')
empt3.head()

,employee_id,name,date_entry,first_country,2nd_country,thrd_count,favorite_word,cell_phone
0,8410,Elaine Hatfield,2019-09-13 22:43:19,United Kingdom (Great Britain),Syria,Côte D'Ivoire (Ivory Coast),elit.,1-420-796-5961
1,7317,Justina Hinton,2019-05-21 21:33:11,Nigeria,Argentina,"Palestine, State of",Integer,1-388-729-3540
2,8771,Charissa Solomon,2018-11-10 00:04:52,Gambia,Guam,Bahrain,lorem,1-429-637-5346
3,5559,Astra Carpenter,2019-07-27 14:12:02,Western Sahara,Benin,Sierra Leone,non,1-567-130-4506
4,7122,Quinn Garcia,2019-10-20 21:40:14,Niue,Afghanistan,Bosnia and Herzegovina,velit.,1-738-469-6123


In [14]:
df2 = pd.merge(left=empt1, right=empt2, on="employee_id", how="left")

In [15]:
df2.head()

,employee_id,date_entry_x,first_country_x,2nd_country_x,thrd_count_x,name,date_entry_y,first_country_y,2nd_country_y,thrd_count_y,favorite_word,cell_phone
0,6755,2020-04-07 00:58:52,"Saint Helena, Ascension and Tristan da Cunha",Palau,Gibraltar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6885,2019-06-02 17:56:06,Georgia,Mongolia,Croatia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8471,2020-05-20 03:33:18,San Marino,Georgia,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7209,2019-07-31 01:42:43,Denmark,Iceland,Tonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5644,2018-11-09 13:59:35,Bolivia,"Virgin Islands, United States",Greece,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df2.shape

(24, 12)

In [17]:
df3 = pd.merge(left=df2, right=empt3, on="employee_id", how="left")

In [18]:
df3.shape

(24, 19)

In [19]:
df3.columns

Index(['employee_id', 'date_entry_x', 'first_country_x', '2nd_country_x', 'thrd_count_x', 'name_x', 'date_entry_y', 'first_country_y', '2nd_country_y', 'thrd_count_y', 'favorite_word_x', 'cell_phone_x', 'name_y', 'date_entry', 'first_country', '2nd_country', 'thrd_count', 'favorite_word_y', 'cell_phone_y'], dtype='object')

In [20]:
df3.head()

,employee_id,date_entry_x,first_country_x,2nd_country_x,thrd_count_x,name_x,date_entry_y,first_country_y,2nd_country_y,thrd_count_y,favorite_word_x,cell_phone_x,name_y,date_entry,first_country,2nd_country,thrd_count,favorite_word_y,cell_phone_y
0,6755,2020-04-07 00:58:52,"Saint Helena, Ascension and Tristan da Cunha",Palau,Gibraltar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6885,2019-06-02 17:56:06,Georgia,Mongolia,Croatia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8471,2020-05-20 03:33:18,San Marino,Georgia,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7209,2019-07-31 01:42:43,Denmark,Iceland,Tonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5644,2018-11-09 13:59:35,Bolivia,"Virgin Islands, United States",Greece,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**==============================================================================================================**

**==============================================================================================================**

**==============================================================================================================**